In [1]:
import sys
sys.executable = '/home/chao/anaconda3/envs/chemcheck_env/lib/python3.8/site-packages'
python_path = ['/home/chao/cantera/build/python', '/home/chao/RMG-Py', '/home/chao', '/home/chao/anaconda3/envs/chemcheck_env/lib/python38.zip', '/home/chao/anaconda3/envs/chemcheck_env/lib/python3.8', '/home/chao/anaconda3/envs/chemcheck_env/lib/python3.8/lib-dynload', '/home/chao/anaconda3/envs/chemcheck_env/lib/python3.8/site-packages']
sys.path.clear()
for path in python_path:
    sys.path.append(path)

In [2]:
import cantera as ct

In [3]:
gas = ct.Solution('gri30.xml')
gas()


  gri30:

       temperature   300 K
          pressure   1.0133e+05 Pa
           density   0.081894 kg/m^3
  mean mol. weight   2.016 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy             26469             53361  J
   internal energy       -1.2108e+06        -2.441e+06  J
           entropy             64910        1.3086e+05  J/K
    Gibbs function       -1.9447e+07       -3.9204e+07  J
 heat capacity c_p             14311             28851  J/K
 heat capacity c_v             10187             20536  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2                 1                 1           -15.717
     [  +52 minor]                 0                 0  



Here I pick reaction 3 to illustrate how to change the forward reaction rates

In [4]:
print("reaction expression:", gas.reaction(2), "k_f:",gas.forward_rate_constants[2], "Delta H:", gas.delta_enthalpy[2])

reaction expression: H2 + O <=> H + OH k_f: 5195.447373103844 Delta H: 8170274.380677491


The block below shows three different enthalpy changes which match to three intervels 
\begin{equation}
Ea = 
\begin{cases}
      0 & \text{for }\Delta H_{rxn} < -4 E_a^0 \\[1em]
     \Delta H_{rxn} & \text{for }\Delta H_{rxn} > 4 E_a^0 \\[1em]
      \frac{\left( w_0 + \frac{\Delta H_{rxn} }{2} \right)  (V_P - 2 w_0 + \Delta H_{rxn}) ^2}{V_P^2 - 4 w_0^2 + \Delta H_{rxn}^2}& \text{otherwise}
    \end{cases}     
\end{equation}
for Blowers Masel approximation. Before call the `forward_rate_constants`, function `add_delta_enthalpy` should be called with input reaction index and changed $\Delta H_{rxn}$. 

In [8]:
delta_h_1 =  gas.delta_enthalpy[2] * 5
delta_h_2 = gas.delta_enthalpy[2] * (-5)
delta_h_3 = gas.delta_enthalpy[2] * 2
gas.add_delta_enthalpy(2, delta_h_1)
print(gas.forward_rate_constants[2])
gas.add_delta_enthalpy(2, delta_h_2)
print(gas.forward_rate_constants[2])
gas.add_delta_enthalpy(2, delta_h_3)
print(gas.forward_rate_constants[2])

52493.11115133174
680485879718.8324
7143603.71754471


The problem so far is that all the changed $\Delta H$s are saved in a new vector, and based on the new factor, it simply re-evaluate the forward rate constants, but other saved properties like Ea for reactions with chagned enthalpy, and equilibrium constant, and reverse rate coefficient are not able to update according to the changed entalpy we input. Also, it only applies for the elementary reactions right now.